# Practical Active Qubit Reset 
This script implement a simulated active qubit reset, as described in the relative [blog post](https://blogs.zhinst.com/andrea/2021/03/05/practical-active-qubit-reset/).


Copyright (C) 2021-2022 Zurich Instruments

This software may be modified and distributed under the terms of the MIT license. See the LICENSE file for details.

Here the connections of the setups, including all the wiring.

![The setup](setup.svg)

In [ ]:
#Setup connectivity
#replace the serial numbers and host with your specifics

#dataserver host
dataserver_host = 'localhost'

# Device names
dev_pqsc = 'dev10000'
dev_hd = 'dev8001'
dev_hd_2 = 'dev8002'
dev_uhf = 'dev2001'

# zsync port used
hd_zsync_port = 1
hd2_zsync_port = 3

In [ ]:
import time
import zhinst.ziPython as zi
import zhinst.utils
import numpy as np
import json
import itertools
import textwrap

daq = zi.ziDAQServer(dataserver_host, 8004, 6)

version = daq.getInt(f'/zi/about/revision')
if version < 220200000:
    raise Exception(f"This script works only with LabOne version 22.02 and greater.")

In [ ]:
def compile_seqc(daq, device, index, program):
    """Compile and send a sequence to the device
    
    Parameters
    ----------
    daq : ziDAQServer 
        The DAQ connection
    device : str
        The serial of the HDAWG
    index: int
        The index of the AWG core. In grouped mode, refers to the first one
    program: str
        The seqc program
    """
    # Setup AWG module
    awg_module = daq.awgModule()
    awg_module.set('device', device)
    awg_module.set('index', index)
    # Execute commands
    awg_module.execute()
    # Compile program
    awg_module.set('compiler/sourcestring', program)
    while awg_module.getInt('compiler/status') == -1:
      time.sleep(0.1)
    if awg_module.getInt('compiler/status') != 0:
        raise Exception("Failed to compile program.")
    # Upload program
    while (awg_module.getDouble('progress') < 1.0) and (awg_module.getInt('elf/status') != 1):
        time.sleep(0.5)
    if awg_module.getInt('elf/status') == 1:
        raise Exception("Failed to upload program.")


def hdawg_config(daq, device, awg_index, program, ct=None, feedback_bit=0):
    """Configure the HDAWG
    
    Parameters
    ----------
    daq : ziDAQServer 
        The DAQ connection
    device : str
        The serial of the HDAWG
    index: int
        The index of the AWG core
    program: str
        The seqc program
    ct: dict
        The Command Table, as dictonary
    feedback_bit: int
        The position of the feedback bit
    """
    
    # Reset the instrument to factory defaults
    daq.set(f'/{device}/system/preset/load', 1)

    # Use ZSync reference clock
    daq.set(f'/{device}/system/clocks/referenceclock/source', 'zsync')
    # Set the correct sample rate
    daq.set(f'/{device}/system/clocks/sampleclock/freq', 2.4e9)

    # Verify that both of the clocks are correctly set
    for _ in range(30):
        if daq.getInt(f'/{device}/system/clocks/referenceclock/status') == 0 and \
           daq.getInt(f'/{device}/system/clocks/sampleclock/status') == 0:
            break
        time.sleep(1)
    else:
        raise Exception(f"Timout while setting the reference and sample clocks")   

    # Configure 4x2 mode
    daq.set(f'/{device}/system/awg/channelgrouping', 'groups_of_2')

    ## Configure DIO

    # LVCMOS interface
    daq.set(f'/{device}/dios/0/interface', 0)
    # DIO mode: QCCS 
    daq.set(f'/{device}/dios/0/mode', 'qccs')
    # Drive the two most significant bytes of the DIO port
    daq.set(f'/{device}/dios/0/drive', 0b1100)

    ## Configure DIO trigger
    # Trigger on any valid input
    daq.set([(f'/{device}/awgs/*/dio/strobe/slope', 'off'),
             (f'/{device}/awgs/*/dio/valid/polarity', 'none')])

    ## Configure AWG
    # Stop AWG
    daq.set(f'/{device}/awgs/{awg_index}/enable', 0)
    # Send sequence
    compile_seqc(daq, device, awg_index, program)
    # Run AWG program once
    daq.set(f'/{device}/awgs/{awg_index}/single', 1)

    # Enable channel outputs
    daq.set(f'/{device}/sigouts/{awg_index*2}/on', 1)
    daq.set(f'/{device}/sigouts/{awg_index*2+1}/on', 1)

    #Configure PQSC data Processing
    daq.set([(f'/{device}/awgs/{awg_index}/zsync/register/shift', feedback_bit),
             (f'/{device}/awgs/{awg_index}/zsync/register/mask', 0b1),
             (f'/{device}/awgs/{awg_index}/zsync/register/offset', 0)])

    #load the command table
    if ct is not None:
        #Create CT 
        ct_all = {'header':{'version':'0.2'}, 'table':ct}
        ct_json_array = np.frombuffer(json.dumps(ct_all).encode('utf-8'), dtype=np.int8)
        daq.set(f"/{device:s}/awgs/{awg_index}/commandtable/data", ct_json_array)

def uhfqa_config(daq, device, program, waves=None, weights=None):
    """Configure the UHFQA
    
    Parameters
    ----------
    daq : ziDAQServer 
        The DAQ connection
    device : str
        The serial of the HDAWG
    program: str
        The seqc program
    waves: list
        List of the waveforms for the AWG
    weights: list
        List of the integrations weights
    """

    # Reset the instrument to factory defaults
    daq.set([(f'/{device}/system/preset/index', 0),
             (f'/{device}/system/preset/load', 1)])

    # Use external reference clock from the HDAWG
    daq.set(f'/{device}/system/extclk', 'external')

    ## Configure DIO
    # Sample DIO data at 50 MHz
    daq.set(f'/{device}/dios/0/extclk', 'internal')
    # Set DIO output to QA result QCCS
    daq.set(f'/{device}/dios/0/mode', 'qa_result_qccs')
    # Drive the two least significant bytes of the DIO port
    daq.set(f'/{device}/dios/0/drive', 0b0011)

    ## Configure DIO triggering 
    # Trigger based on HDAWG DIO protocol
    daq.set([(f'/{device}/awgs/0/dio/strobe/slope', 'off'),
             (f'/{device}/awgs/0/dio/valid/polarity', 'high'),
             (f'/{device}/awgs/0/dio/valid/index', 16)])

    ## Configure AWG
    # Stop AWG
    daq.set(f'{device}/awgs/0/enable', 0)
    # Send sequence
    compile_seqc(daq, device, 0, program)
    # Execute only once (rerun off)
    daq.set(f'/{device}/awgs/0/single', 1)

    #send AWG waves
    waves_set = []
    if waves is not None:
        for i, wave in enumerate(waves):
            wave_raw = zhinst.utils.convert_awg_waveform(wave[0],wave[1])
            waves_set.append((f'/{device}/awgs/0/waveform/waves/{i}', wave_raw))
    daq.set(waves_set)


    #configure QA
    daq.set([(f'/{device}/qas/0/integration/mode', 0),       #Weighted integration mode
             (f'/{device}/qas/0/integration/length', 1024),  #Intgration length
             (f'/{device}/qas/0/thresholds/0/level', 0.0),   #Threshold level
             (f'/{device}/qas/0/delay', 200)])               #Delay depends on the cables

    #Bypass all the non-necessary units
    daq.set([(f'/{device}/qas/0/bypass/deskew',1),
             (f'/{device}/qas/0/bypass/rotation',1),
             (f'/{device}/qas/0/bypass/crosstalk',1)])

    #set weights
    weights_set = []
    if weights is not None:
        for i, weight in enumerate(weights):
            weights_set.append((f'/{device}/qas/0/integration/weights/{i}/real', np.ascontiguousarray(np.real(weight))))
            weights_set.append((f'/{device}/qas/0/integration/weights/{i}/imag', np.ascontiguousarray(np.imag(weight))))
    daq.set(weights_set)

    #configure outputs and inputs
    daq.set([(f'/{device}/awgs/0/outputs/0/amplitude', 1.0),
             (f'/{device}/awgs/0/outputs/1/amplitude', 1.0),

             (f'/{device}/sigouts/0/imp50', 1),
             (f'/{device}/sigouts/1/imp50', 1),
             (f'/{device}/sigins/0/imp50', 1),
             (f'/{device}/sigins/1/imp50', 1),

             (f'/{device}/sigouts/0/on', 1),
             (f'/{device}/sigouts/1/on', 1)])
    
def pqsc_config(daq, device, ports=[0]):
    """Configure the PQSC
    
    Parameters
    ----------
    daq : ziDAQServer 
        The DAQ connection
    device : str
        The serial of the HDAWG
    ports: list
        The ports where the forwarding should be configured
    """

    # Reset the instrument to factory defaults
    daq.set(f'/{device}/system/preset/load', 1)

    # Verify that the PQSC has warmed up and clocks are stable
    if daq.getInt(f'/{device}/system/clocks/ready') != 1:
        raise Exception("PQSC needs to warm up or clock are unstable. Try again later")

    # Use external reference clock. Check if locking is succesful for 30 seconds
    daq.syncSetInt(f'/{device}/system/clocks/referenceclock/in/source', 1)
    for _ in range(30):
        if daq.getInt(f'/{device}/system/clocks/referenceclock/in/status') == 0:
            break
        time.sleep(1)
    if daq.getInt(f'/{device}/system/clocks/referenceclock/in/sourceactual') != 1:
        raise Exception("PQSC failed to lock on external clock. Try again")


    ## Configure execution engine
    # Send a single trigger to start the HDAWG and the UHFQA
    daq.set(f'/{device}/execution/repetitions', 4)
    
    # Wait for feedback to arrive from UHFQA (2us is more than enough)
    daq.set(f'/{device}/execution/holdoff', 2e-6)

    # Clear register bank
    daq.set(f'/{device}/feedback/registerbank/reset', 1)

    ## Register forwarding
    # ZSync output port to the receiver HDAWGs
    for i,port in enumerate(ports):
        # Program register bank forwarding
        daq.set([(f'/{device}/zsyncs/{port}/output/registerbank/sources/0/register', 0),
                 (f'/{device}/zsyncs/{port}/output/registerbank/sources/0/index', i),
                 (f'/{device}/zsyncs/{port}/output/registerbank/sources/0/enable', 1)])

        # Enable forwarding on output port
        daq.set(f'/{device}/zsyncs/{port}/output/registerbank/enable', 1)

In [ ]:
# Connect to PQSC and configure it
daq.connectDevice(dev_pqsc, '1gbe')
pqsc_config(daq, dev_pqsc, ports=[hd_zsync_port, hd2_zsync_port])

In [ ]:
#Connect to the HDAWGs
daq.connectDevice(dev_hd, '1gbe')
daq.connectDevice(dev_hd_2, '1gbe')

#Sequence for the first HDAWG
hd_c1_program = """
//Create one waveform
wave w1 = ones(128);
assignWaveIndex(w1,0);

repeat(4) {
    waitZSyncTrigger();                         //Start trigger
    waitZSyncTrigger();                         //Feedback trigger
    playWaveZSync(ZSYNC_DATA_PQSC_REGISTER);    //Play a waveform depending of feedback data
}
"""

QB1_position = 0   #The position where is the feedback data for QB1

#Command table of first HDAWG
#define two commands, for 0 and 1 feedback values
ct1 = []
ct1.append({'index':0,
            'waveform':{'index':0, 'awgChannel0':['sigout0','sigout1']},
            'amplitude0':{'value':1.0, 'increment': False}})
ct1.append({'index':1,
            'waveform':{'index':0, 'awgChannel0':['sigout0','sigout1']},
            'amplitude0':{'value':0.5, 'increment': False}})

#Sequence for the second HDAWG
hd_c2_program = """
//Create one waveform
wave w1 = ones(512);
assignWaveIndex(w1,0);

repeat(4) {
    waitZSyncTrigger();                         //Start trigger
    waitZSyncTrigger();                         //Feedback trigger
    playWaveZSync(ZSYNC_DATA_PQSC_REGISTER);    //Play a waveform depending of feedback data
}
"""

QB2_position = 0   #The position where is the feedback data for QB2


#Command table of second HDAWG
#define two commands, for 0 and 1 feedback values
#Output on second channel
ct2 = []
ct2.append({'index':0,
            'waveform':{'index':0, 'awgChannel0':['sigout0','sigout1']},
            'amplitude0':{'value':1.0, 'increment': False}})
ct2.append({'index':1,
            'waveform':{'index':0, 'awgChannel0':['sigout0','sigout1']},
            'amplitude0':{'value':0.5, 'increment': False}})


#Compile and send
hdawg_config(daq, dev_hd, 0, hd_c1_program,ct1,QB1_position)
hdawg_config(daq, dev_hd_2, 0, hd_c2_program,ct2,QB2_position)

In [ ]:
"""Envelope function, with gaussian rise/fall and flattop"""
def envelope_function(length, rise, a=1.0, sigmas=3):
    x_risefall = np.arange(-rise,rise,1)
    sigma_wave = rise/sigmas
    risefall_w = np.exp(-(x_risefall)**2/(2*sigma_wave**2))
    length_corr = length - 2*rise
    flattop = np.ones(length_corr)
    res = a*np.concatenate((risefall_w[:rise], flattop, risefall_w[rise:]), axis=None)
    return res

In [ ]:
#Connect to the UHFQA
daq.connectDevice(dev_uhf, '1gbe')

#Define some parametrs of simulated qubit responses
len_wave = 1024
rise = 100

frequencies = [10e6, 31.5e6]
phases = [0, 180]

SR = 1.8e9    #sample rate of UHFQA

channels = len(frequencies)
waves_num = channels * len(phases)
trigger = "|".join(f'QA_INT_{i:d}' for i in range (channels))

readout_reg = 0 #the readout register address

#Sequence for the UHFQA
uhf_program = f'''\
//Define waveforms for readout
//simulated response from a ground and excited state
const len = {len_wave:d};
'''

#Add a waveform placeholder for each simulated response
for i in range(waves_num):
    uhf_program += textwrap.dedent(f"""\
    wave w{i:d}_I = placeholder(len);
    wave w{i:d}_Q = placeholder(len);
    """)

for i in range(waves_num):
    uhf_program += textwrap.dedent(f"""\
    waitZSyncTrigger();                             //Wait for start trigger from PQSC
    playWave(1, w{i:d}_I, 2, w{i:d}_Q);             //play a simulated response
    startQA({trigger:s}, false, {readout_reg:d});   //Trigger a readout
    waitZSyncTrigger();                             //Ignore the feedback
    """)

#Generate integration coefficients
x = np.arange(0,len_wave,1)
envelope = envelope_function(len_wave, rise)
weights = []
for freq in frequencies:
    modcos = np.cos(2*np.pi*freq/SR*x)
    modsin = np.sin(2*np.pi*freq/SR*x)
    
    w = envelope * (modcos + 1j*modsin)
    weights.append(w)

#Generate waves
x = np.arange(0,len_wave,1)
envelope = envelope_function(len_wave, rise)
waves = []

for phs in itertools.product(phases,repeat=channels):
    wave_I = np.zeros(len_wave)
    wave_Q = np.zeros(len_wave)
    
    for freq, ph in zip(frequencies, phs):
        wave_I += np.cos(2*np.pi*freq/SR*x + np.deg2rad(ph))
        wave_Q += np.sin(2*np.pi*freq/SR*x + np.deg2rad(ph))


    wave_I *= envelope/waves_num
    wave_Q *= envelope/waves_num

    waves.append((wave_I, wave_Q))


#Configure the UHFQA and upload sequence, waveforms and weights
uhfqa_config(daq, dev_uhf, uhf_program, waves, weights)

In [ ]:
#Last second check for ZSync clock locking
#Eventually, set again the HDAWGs clock manually to zsync if it failed
def check_zsync_conn(port):
    for _ in range(30):
        if daq.getInt(f'/{dev_pqsc}/zsyncs/{port:d}/connection/status') == 2:
            break
        time.sleep(1)
    else:
        raise Exception(f"Check ZSync connection on HDAWG on port {port+1:d}")

check_zsync_conn(hd_zsync_port)
check_zsync_conn(hd2_zsync_port)

In [ ]:
# Clear register bank before of starting
daq.set(f'/{dev_pqsc}/feedback/registerbank/reset', 1)

def run_stop_awgs(run):
    daq.set([(f'/{dev_hd}/awgs/0/enable', run),
             (f'/{dev_hd_2}/awgs/0/enable', run),
             (f'/{dev_uhf}/awgs/0/enable', run)])

#Stop and then run all the AWG. They will wait the ZSync triggers
run_stop_awgs(False)
run_stop_awgs(True)

#Start the PQSC
daq.set(f'/{dev_pqsc}/execution/enable', 1)